In [6]:
from mne.io import concatenate_raws, read_raw_edf

import os
import mne
import glob
import xlrd
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [82]:
# 获取脑电数据的路径
therapy_path=[
    # r"./患者数据/治疗脑电/第一批次/",
    r"E:/学习资料/papers/数据/人民医院 患者脑电数据/第四批次治疗脑电/同侧耳参考/已发作/",
]
# 切片保存的路径
save_path = "E:/学习资料/papers/数据/人民医院 患者脑电数据/SE切片/"
SE_len = 60 # SE切片长度
frequency = 512 # 采样率，第一批次为1024Hz

In [83]:
# 原始信号
ch_19 = ['EEG Fp1-AV', 'EEG Fp2-AV', 'EEG F3-AV', 'EEG F4-AV', 'EEG C3-AV', 'EEG C4-AV', 
        'EEG P3-AV', 'EEG P4-AV', 'EEG O1-AV', 'EEG O2-AV', 'EEG F7-AV', 'EEG F8-AV',
        'EEG T3-AV', 'EEG T4-AV', 'EEG T5-AV', 'EEG T6-AV','EEG Fz-AV', 'EEG Cz-AV', 
        'EEG Pz-AV']
# 双极纵联
ch_18 = ['EEG Fp1-F3', 'EEG Fp2-F4', 'EEG F3-C3', 'EEG F4-C4', 'EEG C3-P3', 'EEG C4-P4', 
         'EEG P3-O1', 'EEG P4-O2', 'EEG Fp1-F7', 'EEG Fp2-F8', 'EEG F7-T3', 'EEG F8-T4',
         'EEG T3-T5', 'EEG T4-T6', 'EEG T5-O1', 'EEG T6-O2', 'EEG Fz-Cz', 'EEG Cz-Pz']
# 平均参考
ch_17 = ['EEG Fp1-AV', 'EEG Fp2-AV', 'EEG F3-AV', 'EEG F4-AV', 'EEG C3-AV', 'EEG C4-AV', 
            'EEG P3-AV', 'EEG P4-AV', 'EEG O1-AV', 'EEG O2-AV', 'EEG F7-AV', 'EEG F8-AV',
            'EEG T3-AV', 'EEG T4-AV', 'EEG T5-AV', 'EEG T6-AV','EEG Cz-AV']# 抛弃了中央区的两个通道
#同侧耳参考
ch_16= ['EEG Fp1-A1', 'EEG Fp2-A2', 'EEG F3-A1', 'EEG F4-A2', 'EEG C3-A1', 'EEG C4-A2', 
        'EEG P3-A1','EEG P4-A2', 'EEG O1-A1', 'EEG O2-A2', 'EEG F7-A1', 'EEG F8-A2', 
        'EEG T3-A1', 'EEG T4-A2', 'EEG T5-A1', 'EEG T6-A2', 'EEG Fz-AAV', 'EEG Cz-AAV', 'EEG Pz-AAV']

# 选择实验的通道
ch_names = ch_16

In [84]:
# 文件路径；数据采样率；切片长度；通道名；保存目录
def get_se(path,freq,select_time,ch_names,save_path):
    # select_time 采集的时间
    raw=read_raw_edf(path,preload=False)
    fq = freq
    chs = ch_names

    # 选择通道
    raw = raw.pick_channels(chs)
    # 找到SE麻醉标记,save函数的参数单位是s，所以要乘以采样率
    events_from_annot, event_dict = mne.events_from_annotations(raw,verbose='error')
    start_time=events_from_annot[event_dict['SE']-2][0] + 8*freq# 找到SE标签来定位，同时向后延长8s,确保患者进入麻醉状态
    end_time=start_time+select_time*freq # 片段末尾 = 开始时间 + 切片长度(s)*采样率
    print('开始与结束的数据点：',start_time,end_time)
    # 进行切片
    data,times = raw[:,start_time:end_time]
    # 准备生成
    raw_temp=mne.io.RawArray(data,raw.info)
    # 中间部分可以拿到真正的文件名
    path_name=save_path + path.split('\\')[1].split('.')[0] +".fif"    
    path_name=path_name.replace('/','\\')
    raw_temp.save(path_name,overwrite=True)

In [85]:
for i in range(len(therapy_path)):
    therapy_file_path = sorted(glob.glob(therapy_path[i]+"*"))
    for p in therapy_file_path:
        get_se(p,frequency,SE_len,ch_names,save_path[i])

Extracting EDF parameters from E:\学习资料\papers\数据\人民医院 患者脑电数据\第四批次治疗脑电\同侧耳参考\已发作\A12-严聪慧-20220803.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Requested channels: ['EEG Fp1-A1', 'EEG Fp2-A2', 'EEG F3-A1', 'EEG F4-A2', 'EEG C3-A1', 'EEG C4-A2', 'EEG P3-A1', 'EEG P4-A2', 'EEG O1-A1', 'EEG O2-A2', 'EEG F7-A1', 'EEG F8-A2', 'EEG T3-A1', 'EEG T4-A2', 'EEG T5-A1', 'EEG T6-A2', 'EEG Fz-AAV', 'EEG Cz-AAV', 'EEG Pz-AAV']
Available channels in raw: ['EEG Fp1-A1', 'EEG Fp2-A2', 'EEG F3-A1', 'EEG F4-A2', 'EEG C3-A1', 'EEG C4-A2', 'EEG P3-A1', 'EEG P4-A2', 'EEG O1-A1', 'EEG O2-A2', 'EEG F7-A1', 'EEG F8-A2', 'EEG T3-A1', 'EEG T4-A2', 'EEG T5-A1', 'EEG T6-A2', 'EEG Fz-AAV', 'EEG Cz-AAV', 'EEG Pz-AAV']
开始与结束的数据点： 5870 36590
Creating RawArray with float64 data, n_channels=19, n_times=30720
    Range : 0 ... 30719 =      0.000 ...    59.998 secs
Ready.
Writing C:\Users\ANSHANJIAN\EA12-严聪慧-20220803.fif
Closing C:\Users\ANSHANJIAN\EA12-严聪慧-20220803.fif
[done]
Ex

开始与结束的数据点： 39018 69738
Creating RawArray with float64 data, n_channels=19, n_times=30720
    Range : 0 ... 30719 =      0.000 ...    59.998 secs
Ready.
Writing C:\Users\ANSHANJIAN\EA12-何晓丽-20220829.fif
Closing C:\Users\ANSHANJIAN\EA12-何晓丽-20220829.fif
[done]
Extracting EDF parameters from E:\学习资料\papers\数据\人民医院 患者脑电数据\第四批次治疗脑电\同侧耳参考\已发作\A12-何晓丽-20220831.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Requested channels: ['EEG Fp1-A1', 'EEG Fp2-A2', 'EEG F3-A1', 'EEG F4-A2', 'EEG C3-A1', 'EEG C4-A2', 'EEG P3-A1', 'EEG P4-A2', 'EEG O1-A1', 'EEG O2-A2', 'EEG F7-A1', 'EEG F8-A2', 'EEG T3-A1', 'EEG T4-A2', 'EEG T5-A1', 'EEG T6-A2', 'EEG Fz-AAV', 'EEG Cz-AAV', 'EEG Pz-AAV']
Available channels in raw: ['EEG Fp1-A1', 'EEG Fp2-A2', 'EEG F3-A1', 'EEG F4-A2', 'EEG C3-A1', 'EEG C4-A2', 'EEG P3-A1', 'EEG P4-A2', 'EEG O1-A1', 'EEG O2-A2', 'EEG F7-A1', 'EEG F8-A2', 'EEG T3-A1', 'EEG T4-A2', 'EEG T5-A1', 'EEG T6-A2', 'EEG Fz-AAV', 'EEG Cz-AAV', 'EEG Pz-AAV']
开

Closing C:\Users\ANSHANJIAN\EA12-刘锦珍-20220801.fif
[done]
Extracting EDF parameters from E:\学习资料\papers\数据\人民医院 患者脑电数据\第四批次治疗脑电\同侧耳参考\已发作\A12-吴华丽-20220801.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Requested channels: ['EEG Fp1-A1', 'EEG Fp2-A2', 'EEG F3-A1', 'EEG F4-A2', 'EEG C3-A1', 'EEG C4-A2', 'EEG P3-A1', 'EEG P4-A2', 'EEG O1-A1', 'EEG O2-A2', 'EEG F7-A1', 'EEG F8-A2', 'EEG T3-A1', 'EEG T4-A2', 'EEG T5-A1', 'EEG T6-A2', 'EEG Fz-AAV', 'EEG Cz-AAV', 'EEG Pz-AAV']
Available channels in raw: ['EEG Fp1-A1', 'EEG Fp2-A2', 'EEG F3-A1', 'EEG F4-A2', 'EEG C3-A1', 'EEG C4-A2', 'EEG P3-A1', 'EEG P4-A2', 'EEG O1-A1', 'EEG O2-A2', 'EEG F7-A1', 'EEG F8-A2', 'EEG T3-A1', 'EEG T4-A2', 'EEG T5-A1', 'EEG T6-A2', 'EEG Fz-AAV', 'EEG Cz-AAV', 'EEG Pz-AAV']
开始与结束的数据点： 17881 48601
Creating RawArray with float64 data, n_channels=19, n_times=30720
    Range : 0 ... 30719 =      0.000 ...    59.998 secs
Ready.
Writing C:\Users\ANSHANJIAN\EA12-吴华丽-20220801.fif
C

Creating raw.info structure...
Requested channels: ['EEG Fp1-A1', 'EEG Fp2-A2', 'EEG F3-A1', 'EEG F4-A2', 'EEG C3-A1', 'EEG C4-A2', 'EEG P3-A1', 'EEG P4-A2', 'EEG O1-A1', 'EEG O2-A2', 'EEG F7-A1', 'EEG F8-A2', 'EEG T3-A1', 'EEG T4-A2', 'EEG T5-A1', 'EEG T6-A2', 'EEG Fz-AAV', 'EEG Cz-AAV', 'EEG Pz-AAV']
Available channels in raw: ['EEG Fp1-A1', 'EEG Fp2-A2', 'EEG F3-A1', 'EEG F4-A2', 'EEG C3-A1', 'EEG C4-A2', 'EEG P3-A1', 'EEG P4-A2', 'EEG O1-A1', 'EEG O2-A2', 'EEG F7-A1', 'EEG F8-A2', 'EEG T3-A1', 'EEG T4-A2', 'EEG T5-A1', 'EEG T6-A2', 'EEG Fz-AAV', 'EEG Cz-AAV', 'EEG Pz-AAV']
开始与结束的数据点： 23351 54071
Creating RawArray with float64 data, n_channels=19, n_times=30720
    Range : 0 ... 30719 =      0.000 ...    59.998 secs
Ready.
Writing C:\Users\ANSHANJIAN\EA12-唐心诚-20220905.fif
Closing C:\Users\ANSHANJIAN\EA12-唐心诚-20220905.fif
[done]
Extracting EDF parameters from E:\学习资料\papers\数据\人民医院 患者脑电数据\第四批次治疗脑电\同侧耳参考\已发作\A12-夏帆-20220914.edf...
EDF file detected
Setting channel info structure...
Cr

    Range : 0 ... 30719 =      0.000 ...    59.998 secs
Ready.
Writing C:\Users\ANSHANJIAN\EA12-尤李心怡-20220824.fif
Closing C:\Users\ANSHANJIAN\EA12-尤李心怡-20220824.fif
[done]
Extracting EDF parameters from E:\学习资料\papers\数据\人民医院 患者脑电数据\第四批次治疗脑电\同侧耳参考\已发作\A12-张会-20220713.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Requested channels: ['EEG Fp1-A1', 'EEG Fp2-A2', 'EEG F3-A1', 'EEG F4-A2', 'EEG C3-A1', 'EEG C4-A2', 'EEG P3-A1', 'EEG P4-A2', 'EEG O1-A1', 'EEG O2-A2', 'EEG F7-A1', 'EEG F8-A2', 'EEG T3-A1', 'EEG T4-A2', 'EEG T5-A1', 'EEG T6-A2', 'EEG Fz-AAV', 'EEG Cz-AAV', 'EEG Pz-AAV']
Available channels in raw: ['EEG Fp1-A1', 'EEG Fp2-A2', 'EEG F3-A1', 'EEG F4-A2', 'EEG C3-A1', 'EEG C4-A2', 'EEG P3-A1', 'EEG P4-A2', 'EEG O1-A1', 'EEG O2-A2', 'EEG F7-A1', 'EEG F8-A2', 'EEG T3-A1', 'EEG T4-A2', 'EEG T5-A1', 'EEG T6-A2', 'EEG Fz-AAV', 'EEG Cz-AAV', 'EEG Pz-AAV']
开始与结束的数据点： 7082 37802
Creating RawArray with float64 data, n_channels=19, n_times=30720
 

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Requested channels: ['EEG Fp1-A1', 'EEG Fp2-A2', 'EEG F3-A1', 'EEG F4-A2', 'EEG C3-A1', 'EEG C4-A2', 'EEG P3-A1', 'EEG P4-A2', 'EEG O1-A1', 'EEG O2-A2', 'EEG F7-A1', 'EEG F8-A2', 'EEG T3-A1', 'EEG T4-A2', 'EEG T5-A1', 'EEG T6-A2', 'EEG Fz-AAV', 'EEG Cz-AAV', 'EEG Pz-AAV']
Available channels in raw: ['EEG Fp1-A1', 'EEG Fp2-A2', 'EEG F3-A1', 'EEG F4-A2', 'EEG C3-A1', 'EEG C4-A2', 'EEG P3-A1', 'EEG P4-A2', 'EEG O1-A1', 'EEG O2-A2', 'EEG F7-A1', 'EEG F8-A2', 'EEG T3-A1', 'EEG T4-A2', 'EEG T5-A1', 'EEG T6-A2', 'EEG Fz-AAV', 'EEG Cz-AAV', 'EEG Pz-AAV']
开始与结束的数据点： 18900 49620
Creating RawArray with float64 data, n_channels=19, n_times=30720
    Range : 0 ... 30719 =      0.000 ...    59.998 secs
Ready.
Writing C:\Users\ANSHANJIAN\EA12-成思思-20220803.fif
Closing C:\Users\ANSHANJIAN\EA12-成思思-20220803.fif
[done]
Extracting EDF parameters from E:\学习资料\papers\数据\人民医院 患者脑电数据\第四批次治疗脑电\同侧耳参考\已发作\A12-成思思-20220805.edf...
E

    Range : 0 ... 30719 =      0.000 ...    59.998 secs
Ready.
Writing C:\Users\ANSHANJIAN\EA12-王云芝-20220720.fif
Closing C:\Users\ANSHANJIAN\EA12-王云芝-20220720.fif
[done]
Extracting EDF parameters from E:\学习资料\papers\数据\人民医院 患者脑电数据\第四批次治疗脑电\同侧耳参考\已发作\A12-王敏敏-20220826.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Requested channels: ['EEG Fp1-A1', 'EEG Fp2-A2', 'EEG F3-A1', 'EEG F4-A2', 'EEG C3-A1', 'EEG C4-A2', 'EEG P3-A1', 'EEG P4-A2', 'EEG O1-A1', 'EEG O2-A2', 'EEG F7-A1', 'EEG F8-A2', 'EEG T3-A1', 'EEG T4-A2', 'EEG T5-A1', 'EEG T6-A2', 'EEG Fz-AAV', 'EEG Cz-AAV', 'EEG Pz-AAV']
Available channels in raw: ['EEG Fp1-A1', 'EEG Fp2-A2', 'EEG F3-A1', 'EEG F4-A2', 'EEG C3-A1', 'EEG C4-A2', 'EEG P3-A1', 'EEG P4-A2', 'EEG O1-A1', 'EEG O2-A2', 'EEG F7-A1', 'EEG F8-A2', 'EEG T3-A1', 'EEG T4-A2', 'EEG T5-A1', 'EEG T6-A2', 'EEG Fz-AAV', 'EEG Cz-AAV', 'EEG Pz-AAV']
开始与结束的数据点： 17892 48612
Creating RawArray with float64 data, n_channels=19, n_times=30720
 

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Requested channels: ['EEG Fp1-A1', 'EEG Fp2-A2', 'EEG F3-A1', 'EEG F4-A2', 'EEG C3-A1', 'EEG C4-A2', 'EEG P3-A1', 'EEG P4-A2', 'EEG O1-A1', 'EEG O2-A2', 'EEG F7-A1', 'EEG F8-A2', 'EEG T3-A1', 'EEG T4-A2', 'EEG T5-A1', 'EEG T6-A2', 'EEG Fz-AAV', 'EEG Cz-AAV', 'EEG Pz-AAV']
Available channels in raw: ['EEG Fp1-A1', 'EEG Fp2-A2', 'EEG F3-A1', 'EEG F4-A2', 'EEG C3-A1', 'EEG C4-A2', 'EEG P3-A1', 'EEG P4-A2', 'EEG O1-A1', 'EEG O2-A2', 'EEG F7-A1', 'EEG F8-A2', 'EEG T3-A1', 'EEG T4-A2', 'EEG T5-A1', 'EEG T6-A2', 'EEG Fz-AAV', 'EEG Cz-AAV', 'EEG Pz-AAV']
开始与结束的数据点： 36595 67315
Creating RawArray with float64 data, n_channels=19, n_times=30720
    Range : 0 ... 30719 =      0.000 ...    59.998 secs
Ready.
Writing C:\Users\ANSHANJIAN\EA12-罗文彤-20220720.fif
Closing C:\Users\ANSHANJIAN\EA12-罗文彤-20220720.fif
[done]
Extracting EDF parameters from E:\学习资料\papers\数据\人民医院 患者脑电数据\第四批次治疗脑电\同侧耳参考\已发作\A12-罗文彤-20220722.edf...
E

开始与结束的数据点： 25833 56553
Creating RawArray with float64 data, n_channels=19, n_times=30720
    Range : 0 ... 30719 =      0.000 ...    59.998 secs
Ready.
Writing C:\Users\ANSHANJIAN\EA12-胡杨-20220815.fif
Closing C:\Users\ANSHANJIAN\EA12-胡杨-20220815.fif
[done]
Extracting EDF parameters from E:\学习资料\papers\数据\人民医院 患者脑电数据\第四批次治疗脑电\同侧耳参考\已发作\A12-胡杨-20220819.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Requested channels: ['EEG Fp1-A1', 'EEG Fp2-A2', 'EEG F3-A1', 'EEG F4-A2', 'EEG C3-A1', 'EEG C4-A2', 'EEG P3-A1', 'EEG P4-A2', 'EEG O1-A1', 'EEG O2-A2', 'EEG F7-A1', 'EEG F8-A2', 'EEG T3-A1', 'EEG T4-A2', 'EEG T5-A1', 'EEG T6-A2', 'EEG Fz-AAV', 'EEG Cz-AAV', 'EEG Pz-AAV']
Available channels in raw: ['EEG Fp1-A1', 'EEG Fp2-A2', 'EEG F3-A1', 'EEG F4-A2', 'EEG C3-A1', 'EEG C4-A2', 'EEG P3-A1', 'EEG P4-A2', 'EEG O1-A1', 'EEG O2-A2', 'EEG F7-A1', 'EEG F8-A2', 'EEG T3-A1', 'EEG T4-A2', 'EEG T5-A1', 'EEG T6-A2', 'EEG Fz-AAV', 'EEG Cz-AAV', 'EEG Pz-AAV']
开始与结

[done]
Extracting EDF parameters from E:\学习资料\papers\数据\人民医院 患者脑电数据\第四批次治疗脑电\同侧耳参考\已发作\A12-谢思沂-20220715.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Requested channels: ['EEG Fp1-A1', 'EEG Fp2-A2', 'EEG F3-A1', 'EEG F4-A2', 'EEG C3-A1', 'EEG C4-A2', 'EEG P3-A1', 'EEG P4-A2', 'EEG O1-A1', 'EEG O2-A2', 'EEG F7-A1', 'EEG F8-A2', 'EEG T3-A1', 'EEG T4-A2', 'EEG T5-A1', 'EEG T6-A2', 'EEG Fz-AAV', 'EEG Cz-AAV', 'EEG Pz-AAV']
Available channels in raw: ['EEG Fp1-A1', 'EEG Fp2-A2', 'EEG F3-A1', 'EEG F4-A2', 'EEG C3-A1', 'EEG C4-A2', 'EEG P3-A1', 'EEG P4-A2', 'EEG O1-A1', 'EEG O2-A2', 'EEG F7-A1', 'EEG F8-A2', 'EEG T3-A1', 'EEG T4-A2', 'EEG T5-A1', 'EEG T6-A2', 'EEG Fz-AAV', 'EEG Cz-AAV', 'EEG Pz-AAV']
开始与结束的数据点： 55817 86537
Creating RawArray with float64 data, n_channels=19, n_times=30720
    Range : 0 ... 30719 =      0.000 ...    59.998 secs
Ready.
Writing C:\Users\ANSHANJIAN\EA12-谢思沂-20220715.fif
Closing C:\Users\ANSHANJIAN\EA12-谢思沂-20220715.fif
[